# Titulo

Este verano del 2023 viene siendo muy intenso para la mayoría de gente. He escuchado y leído por las redes sociales a muchas personas decir que este verano lo sienten como el más caluroso de los últimos años.

Sin embargo, me percaté de que cada año que pasa, la gente piensa esto mismo nuevamente. Por lo que me propuse buscar los datos históricos de las temperaturas medias de Buenos Aires y hacer un análisis.  

Pude encontrar un dataset considerablemente completo, con un historial de temperaturas registradas provincia por provincia de Argentina en una web del gobierno en [este link](https://datos.gob.ar/dataset/smn-estadisticas-climaticas-normales). Pero desafortunadamente el registro llegaba hasta el año 2010 por lo que no me es de gran utilidad.

Luego encontré en una web de la Ciudad este dataset con las temperaturas máximas y mínimas medias de cada mes entre 1991 y 2023, el más actualizado hasta la actualidad. Por lo tanto, ante la imposibilidad de encontrar información suficiente sobre las temperaturas históricas de la provincia de Buenos Aires, me decidí por realizar el proyecto con el registro de CABA. [Fuente aquí](https://www.estadisticaciudad.gob.ar/eyc/?p=27702).

## Data parsing

Uso la biblioteca `xlrd` para leer archivos de Excel

In [ ]:
!pip install xlrd

In [ ]:
import xlrd
import json

workbook = xlrd.open_workbook("MA_AX01.xls")
sheet = workbook.sheet_by_index(0) # obtengo la primer hoja

Ahora inicializo un diccionario para tener la info de cada año. Me quedaré por ahora con los meses de Diciembre, Enero y Febrero, que tienden a ser los meses más calurosos del verano antes de que este acabe en Marzo, y asignaré las temperaturas máximas y mínimas medias en forma de lista de dos elementos (max,min). Como el verano abarca el fin de un mes y el principio de otro, para simplificar cuentas a un año le asignare su Enero y Febrero, y el Diciembre del mes anterior. Discriminaré el 1991 pues no tengo la info de su verano completa.

In [ ]:
dataClima = {}
            
for año in range (1992, 2024):
    dataClima[año] = {"Dic": [0,0], "Ene": [0,0], "Feb": [0,0]}

A partir del año 2018 comenzaron a incluir la temperatura media de cada mes, pero no la tomaré. Primero lleno el datClima con los años 1991 - 2017, y  luego del 2018-2023. Las columnas con las temepraturas medias van de 2 a 54 y las filas de cada mes son 4,5 y 15

In [ ]:
columna = 3

for año in range (1992,2018):
    dataClima[año]["Dic"] = [sheet.cell_value(14,columna-2), sheet.cell_value(14,columna-1)]
    dataClima[año]["Ene"] = [sheet.cell_value(3,columna), sheet.cell_value(3,columna+1)]
    dataClima[año]["Feb"] = [sheet.cell_value(4,columna), sheet.cell_value(4,columna+1)]
        
    columna += 2

# Arreglo los indices de columna por el desfase a partir del 2018
    
columna += 1
dataClima[2018]["Dic"] = [sheet.cell_value(14,columna-3), sheet.cell_value(14,columna-2)]
dataClima[2018]["Ene"] = [sheet.cell_value(3,columna), sheet.cell_value(3,columna+1)]
dataClima[2018]["Feb"] = [sheet.cell_value(4,columna), sheet.cell_value(4,columna+1)]


columna += 3
        
for año in range (2019, 2024):
    dataClima[año]["Dic"] = [sheet.cell_value(14,columna-3), sheet.cell_value(14,columna-2)]
    dataClima[año]["Ene"] = [sheet.cell_value(3,columna), sheet.cell_value(3,columna+1)]
    dataClima[año]["Feb"] = [sheet.cell_value(4,columna), sheet.cell_value(4,columna+1)]
    
    columna += 3
    
# print (json.dumps(dataClima, indent=4))

## Analizo los datos obtenidos

Ahora obtendré la media de las temperaturas máximas y mínimas medias de cada verano, y ordenaré los veranos según cuán calurosos y cuán frios fueron.

In [17]:
tempsMediasDeVeranos = []

def calcTempsMedia (añoNum, añoDict): # recibe cada diccionario dentro del diccionario dataClima
    mediaMax = (añoDict["Dic"][0] + añoDict["Ene"][0] + añoDict["Feb"][0])/3
    mediaMin = (añoDict["Dic"][1] + añoDict["Ene"][1] + añoDict["Feb"][1])/3
    
    return (añoNum,(mediaMax, mediaMin))

for año in range (1992, 2024):
    duplaAñoMedias = calcTempsMedia(año, dataClima[año])
    tempsMediasDeVeranos.append(duplaAñoMedias)
    
print(tempsMediasDeVeranos)

[(1992, (28.433333333333334, 19.433333333333334)), (1993, (28.46666666666667, 19.633333333333333)), (1994, (27.7, 18.733333333333334)), (1995, (29.5, 19.466666666666665)), (1996, (29.7, 19.433333333333334)), (1997, (29.833333333333332, 20.1)), (1998, (26.7, 17.8)), (1999, (27.899999999999995, 18.166666666666668)), (2000, (29.833333333333332, 19.866666666666664)), (2001, (29.633333333333336, 19.599999999999998)), (2002, (28.566666666666666, 18.633333333333336)), (2003, (28.833333333333332, 18.9)), (2004, (28.566666666666663, 18.166666666666668)), (2005, (29.3, 19.566666666666666)), (2006, (28.73333333333333, 18.633333333333333)), (2007, (29.7, 19.833333333333332)), (2008, (29.933333333333337, 19.766666666666666)), (2009, (30.100000000000005, 19.76666666666667)), (2010, (28.766666666666666, 19.733333333333334)), (2011, (29.933333333333337, 19.966666666666665)), (2012, (29.96666666666667, 19.366666666666664)), (2013, (29.76666666666667, 19.5)), (2014, (30.233333333333334, 20.1333333333333